In [ ]:
!pip install hydra-core==1.1

In [2]:
!pip install scipy==1.7.3

BRANCH = 'main'
# If you're using Google Colab and not running locally, uncomment and run this cell.
!apt-get install sox libsndfile1 ffmpeg
!pip install wget unidecode pynini==2.1.4
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

#=== load the repo and data (Thanks Synthbot) ===
!sudo dpkg --configure -a
!sudo apt -qq install -y sox
!git clone "https://github.com/synthbot-anon/synthbot.git" /content/synthbot
!(cd /content/synthbot; git checkout experimental)
!python3 -m pip install pysoundfile 
!python3 -m pip install librosa

!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Get:1 http://archive

In [ ]:
#imports
import os
import json
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
import soundfile as sf
import numpy as np

#helper functions
def infer(spec_gen_model, vocoder_model, str_input, speaker=None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def custom_infer(transcript, spec_model, vocoder, download=None):
  spec, audio = infer(spec_model, vocoder, transcript)
  print("\n\n\n\n")
  ipd.display(ipd.Audio(audio, rate=22050))
  %matplotlib inline
  plt.show()

def try_infering_with_finetuned_fastpitch(checkpointpath, sentence):
  spec_model = FastPitchModel.load_from_checkpoint(checkpointpath)
  spec_model.eval().cuda()

  vocoder = HifiGanModel.from_pretrained("tts_hifigan")
  vocoder = vocoder.eval().cuda()

  custom_infer(sentence, spec_model, vocoder)

In [ ]:
# additional files
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b_nv22.07 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921 \
&& wget wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/nemo_text_processing/text_normalization/en/data/whitelist/lj_speech.tsv \
&& cd ..

In [47]:
checkpoint = '/content/drive/MyDrive/TTS_finetuning/trixie_02/FastPitch--v_loss=1.4934-epoch=496-last.ckpt'

In [48]:
spec_model = FastPitchModel.load_from_checkpoint(checkpoint)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

[NeMo I 2022-07-28 20:35:19 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-28 20:35:41 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-28 20:35:42 modelPT:143] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: fastpitch_train.json
      sample_rate: 22050
      sup_data_path: fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_duration: 0.1
      ignore_file: null
      

[NeMo I 2022-07-28 20:35:42 features:217] PADDING: 1
[NeMo I 2022-07-28 20:35:42 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-28 20:35:42 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-28 20:35:42 common:910] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-28 20:35:45 modelPT:143] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-28 20:35:45 modelPT:150] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-28 20:35:45 features:217] PADDING: 0


[NeMo W 2022-07-28 20:35:45 features:195] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-28 20:35:45 features:217] PADDING: 0
[NeMo I 2022-07-28 20:35:46 save_restore_connector:243] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [56]:
custom_infer("If the rise of an all-powerful artificial intelligence is inevitable, well it stands to reason that when they take power, our digital overlords will punish those of us who did not help them get there. Ergo, I would like to be a helpful idiot. Like yourself.", spec_model, vocoder)

In [57]:
custom_infer("It’s not magic, it’s talent and sweat. People like me ensure your packets get delivered unsniffed. So what do I do? I make sure that one bad config on one key component doesn’t bankrupt the entire fucking company.", spec_model, vocoder)

In [58]:
custom_infer("Tell me and I forget. Teach me and I remember. Involve me and I learn.", spec_model, vocoder)

In [59]:
custom_infer("They may take our lives, but they'll never take our freedom!", spec_model, vocoder)

In [60]:
custom_infer("I'm gonna make him an offer he can't refuse.", spec_model, vocoder)